In [1]:
%cd ..
%pwd
# should be /code

c:\Users\Oliver\Research\agents-goals-pressure\code


'c:\\Users\\Oliver\\Research\\agents-goals-pressure\\code'

In [2]:
%load_ext autoreload
%autoreload 2

In [12]:
from pgmpy.models import BayesianNetwork,  FactorGraph

from pgmpy.readwrite import BIFReader
reader = BIFReader('data/alarm.bif')
#reader.get_edges()

In [13]:
alarm = reader.get_model()

In [13]:
M.graph

{'name': 'Unknown'}

In [6]:
from pgmpy.factors.discrete import TabularCPD

In [7]:
from pdg import PDG
from pdg.rv import Variable as Var
from pdg.dist import CPT

M = PDG()
A = Var.alph("A",3);
B = Var.alph("B",2);
D = Var.alph("D",5)
M += A, B, D

In [8]:
P = CPT.make_random(A & D, B); M += P
P

c:\Users\Oliver\Research\agents-goals-pressure\code\pdg\dist.py:199: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  self /= np.sum(self, axis=1)[:, None]


,b0,b1
"(a0, d0)",0.352160,0.647840
"(a0, d1)",0.219854,0.780146
"(a0, d2)",0.450810,0.549190
"(a0, d3)",0.518021,0.481979
"(a0, d4)",0.465549,0.534451
"(a1, d0)",0.454942,0.545058
"(a1, d1)",0.641555,0.358445
"(a1, d2)",0.021953,0.978047
"(a1, d3)",0.283502,0.716498
"(a1, d4)",0.346866,0.653134


In [9]:
tcpd = TabularCPD(P.nto.name, len(P.nto), P.to_numpy().T, [a.name for a in P.nfrom.atoms], [len(v) for v in P.nfrom.atoms])
tcpd.get_value(B=0, D=0, A=0)

0.3521603672420116

In [77]:
print(P.as_(TabularCPD))

+-------+---------------------+--------------------+---------------------+
| A     | A(a0)               | A(a1)              | A(a2)               |
+-------+---------------------+--------------------+---------------------+
| B(b0) | 0.09701251401987165 | 0.5779129965171199 | 0.19557075362866636 |
+-------+---------------------+--------------------+---------------------+
| B(b1) | 0.9029874859801283  | 0.4220870034828801 | 0.8044292463713336  |
+-------+---------------------+--------------------+---------------------+


In [78]:
P

,b0,b1
a0,0.097013,0.902987
a1,0.577913,0.422087
a2,0.195571,0.804429


**Testing what happens if you add cycles / multiple cpds on same node**...

In [82]:
from pgmpy.models import BayesianNetwork

bn = BayesianNetwork([('A', 'B'), ('D','B')])


... Doesn't work.

In [74]:
tcpd.values.shape, np.moveaxis(tcpd.values,0,-1).shape, P.shape

((2, 3, 5), (3, 5, 2), (15, 2))

In [62]:
print(P.index, P.index[0])
P.loc[[('a0', 'd0')]] ## Note: need extra pair of brackets because of nonsense...

Index([('a0', 'd0'), ('a0', 'd1'), ('a0', 'd2'), ('a0', 'd3'), ('a0', 'd4'),
       ('a1', 'd0'), ('a1', 'd1'), ('a1', 'd2'), ('a1', 'd3'), ('a1', 'd4'),
       ('a2', 'd0'), ('a2', 'd1'), ('a2', 'd2'), ('a2', 'd3'), ('a2', 'd4')],
      dtype='object') ('a0', 'd0')


,b0,b1
"(a0, d0)",0.211105,0.788895


In [10]:
print(P.to_pgmpy())

+-------+--------------------+-----+--------------------+
| A     | A(a0)              | ... | A(a2)              |
+-------+--------------------+-----+--------------------+
| D     | D(d0)              | ... | D(d4)              |
+-------+--------------------+-----+--------------------+
| B(b0) | 0.3521603672420116 | ... | 0.1345429665176714 |
+-------+--------------------+-----+--------------------+
| B(b1) | 0.6478396327579884 | ... | 0.8654570334823286 |
+-------+--------------------+-----+--------------------+


In [16]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
student = BayesianNetwork([('diff', 'grades'), ('intel', 'grades')])
grades_cpd = TabularCPD('grades', 5, [[0.1,0.1,0.1,0.1,0.1,0.1],
                                      [0.1,0.1,0.1,0.1,0.1,0.1],
                                      [0.7,0.7,0.7,0.7,0.7,0.7],
                                      [.09,.09,.09,.09,.09,0.1],
                                      [.01,.01,.01,.01,.01,0.0]],
                        evidence=['diff', 'intel'], evidence_card=[2, 3])
student.add_cpds(grades_cpd)

In [19]:
grades_cpd.cardinality

array([5, 2, 3])

In [21]:
CPT.from_pgmpy(grades_cpd)

c:\Users\Oliver\Research\agents-goals-pressure\code\pdg\dist.py:194: UserWarning: 0.02-Unnormalized CPT
  warnings.warn("%.2f-Unnormalized CPT"%amt)


,0,1,2,3,4
"(0, 0)",0.1,0.1,0.7,0.05,0.00
"(0, 1)",0.1,0.1,0.7,0.05,0.00
"(0, 2)",0.1,0.1,0.7,0.05,0.00
"(1, 0)",0.1,0.1,0.7,0.05,0.00
"(1, 1)",0.1,0.1,0.7,0.05,0.00
"(1, 2)",0.1,0.1,0.7,0.00,0.05
